In [1]:
# Import libraries
from google.cloud import bigquery
from google.cloud import bigquery_storage
import pandas as pd
import plotly.express as px

# Read project from env variable
from dotenv import load_dotenv
import os
load_dotenv()  # Loads .env file into environment variables
project_id = os.environ.get("DSAI_PROJECT_ID")

# Initialize BigQuery client
client = bigquery.Client(project=project_id)
bqstorage_client = bigquery_storage.BigQueryReadClient()

query_check = f"""
SELECT trip_start, COUNT(*) AS cnt
FROM `{project_id}.LondonBicycles.fact_trips`
GROUP BY trip_start
ORDER BY trip_start DESC
LIMIT 10
"""
df_check = client.query(query_check).to_dataframe()
print(df_check)

# --------------------------------------------
# Step 1: Read fact_trips table from BigQuery
# --------------------------------------------
query_fact = f"""
SELECT
    rental_id,
    trip_start,
    trip_end,
    bike_id,
    start_station_id,
    end_station_id,
    TIMESTAMP_MICROS(trip_start) AS start_ts
FROM `{project_id}.LondonBicycles.fact_trips`
WHERE trip_start > 0 AND trip_start <= 2534023007999999999
"""

df_trips = client.query(query_fact).to_dataframe(bqstorage_client=bqstorage_client)
print("Fact trips rows:", len(df_trips))

# --------------------------------------------
# Step 2: Read dim_stations table
# --------------------------------------------
query_stations = f"""
SELECT
    station_id,
    station_name
FROM `{project_id}.LondonBicycles.dim_stations`
"""

df_stations = client.query(query_stations).to_dataframe()
print("Stations rows:", len(df_stations))

# --------------------------------------------
# Step 3: Join trips with start station names
# --------------------------------------------
df_trips = df_trips.merge(df_stations, left_on="start_station_id", right_on="station_id", how="left")
df_trips.rename(columns={"station_name": "start_station_name"}, inplace=True)

# --------------------------------------------
# Step 4: Add derived columns for visualization
# --------------------------------------------
df_trips["hour"] = df_trips["start_ts"].dt.hour
df_trips["weekday"] = df_trips["start_ts"].dt.dayofweek + 1  # Monday=1, Sunday=7

# --------------------------------------------
# Step 5: Aggregate trips by hour (polar plot)
# --------------------------------------------
df_hour = df_trips.groupby("hour").size().reset_index(name="trip_count")

fig_hour = px.line_polar(
    df_hour, r="trip_count", theta="hour", line_close=True,
    title="London Bikeshare Trips by Hour"
)
fig_hour.show()

# --------------------------------------------
# Step 6: Aggregate trips by weekday (polar plot)
# --------------------------------------------
df_weekday = df_trips.groupby("weekday").size().reset_index(name="trip_count")

fig_weekday = px.line_polar(
    df_weekday, r="trip_count", theta="weekday", line_close=True,
    title="London Bikeshare Trips by Weekday"
)
fig_weekday.show()

# --------------------------------------------
# Step 7: Optional - Trips by station and hour (heatmap)
# --------------------------------------------
df_station_hour = df_trips.groupby(["start_station_name", "hour"]).size().reset_index(name="trip_count")

fig_station_hour = px.density_heatmap(
    df_station_hour, x="hour", y="start_station_name", z="trip_count",
    title="Trips by Station and Hour", color_continuous_scale="Viridis"
)
fig_station_hour.show()


         trip_start  cnt
0  1673827140000000    4
1  1673827080000000    2
2  1673827020000000   11
3  1673826960000000    3
4  1673826900000000    1
5  1673826840000000    4
6  1673826780000000    2
7  1673826720000000    3
8  1673826660000000    3
9  1673826600000000    2
Fact trips rows: 83434866
Stations rows: 800
